In [40]:
import pandas as pd
import string
import os
import numpy as np
from sqlalchemy import create_engine, text

from backend.utils import DbEngine, Queries
from backend.etl import ETL
from backend.etl_sdhi import ETL_SDHI
from encoding_module.encoder import Encoding

In [23]:
! pip install unidecode

     |████████████████████████████████| 238 kB 2.0 MB/s eta 0:00:01


### Connect to DB

In [25]:
db_engine = DbEngine(user = 'team77', 
                     password = 'mintic2020.',
                     ip = 'postgres', 
                     port = '5432', 
                     db = 'minjusticia')
engine = db_engine.connect()
queries = Queries(engine)

### Loading original data

In [26]:
data = pd.read_excel('../data/ReincidenciaPospenadosNal201011Junio2020Rev.xlsx', skiprows = 6)


In [33]:
data2 = data[['INTERNOEN', 'DELITO', 'TITULO_DELITO', 'SUBTITULO_DELITO', 'TENTATIVA',
       'AGRAVADO', 'CALIFICADO', 'FECHA_INGRESO', 'FECHA_SALIDA',
       'FECHA_CAPTURA', 'SITUACION_JURIDICA', 'ANO_NACIMIENTO', 'EDAD',
       'GENERO', 'ESTADO_CIVIL', 'PAIS_INTERNO', 'DEPARTAMENTO', 'CIUDAD',
       'REINCIDENTE', 'ESTADO_INGRESO', 'ACTIVIDADES_TRABAJO',
       'ACTIVIDADES_ESTUDIO', 'ACTIVIDADES_ENSEÑANZA', 'NIVEL_EDUCATIVO',
       'HIJOS_MENORES', 'CONDIC_EXPECIONAL', 'CODIGO_ESTABLECIMIENTO',
       'ESTABLECIMIENTO', 'DEPTO_ESTABLECIMIENTO', 'MPIO_ESTABLECIMIENTO',
       'REGIONAL', 'ESTADO']]
data2['AGRAVADO'].unique()

array(['N', 'S'], dtype=object)

### Execute ETL

In [41]:
etl = ETL(queries)
etl.inmate_static_info(data2)
etl.inmate_variable_info(data2)

/home/jovyan/work/esneyder/Documents/ciencia de datos 2020/proyect_minjusticia/MinJusticia_team77_DS4A_Project/app/backend/etl.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reg['CONDIC_EXPECIONAL'] = data_reg['CONDIC_EXPECIONAL'].fillna('NINGUNO')
/home/jovyan/work/esneyder/Documents/ciencia de datos 2020/proyect_minjusticia/MinJusticia_team77_DS4A_Project/app/backend/etl.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_reg['madre_gestante'] = 'NA'
/opt/conda/lib/python3.7/site-packages

,crime,min_month,max_month,decree
0,HURTO,16,108,239
1,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,128,360,376
2,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,132,180,366
3,CONCIERTO PARA DELINQUIR,48,108,340
4,HOMICIDIO,208,450,103
5,EXTORSION,192,288,244
6,"FABRICACIÓN, TRÁFICO, PORTE O TENENCIA DE ARMA...",108,144,365
7,VIOLENCIA INTRAFAMILIAR,48,720,229
8,LESIONES PERSONALES,16,360,"[111, 112, 113, 114, 115, 116, 117, 118, 119, ..."
9,FABRICACION TRAFICO Y PORTE DE ARMAS Y MUNICI...,132,180,366


,INTERNOEN,DELITO,TITULO_DELITO,SUBTITULO_DELITO,TENTATIVA,AGRAVADO,CALIFICADO,FECHA_INGRESO,FECHA_SALIDA,FECHA_CAPTURA,...,REGIONAL,ESTADO,crime_score,CONSUMADO_b,TENTATIVA_b,CALIFICADO_b,AGRAVADO_b,AGRAVADO_b,crime_multiplier,severity
0,5496C737C624136DB406CFC6744C684C070A3CD8,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,CONTRA LA SEGURIDAD PUBLICA,DE LOS DELITOS DE PELIGRO COMUN O QUE PUEDEN O...,N,N,N,01/01/10,12/05/11,30/12/09,...,NOROESTE,Baja,4,4,0,0,0,0,0.571429,2.285714


,TENTATIVA,AGRAVADO,CALIFICADO,multiplier
0,N,N,N,2
1,N,S,S,8
2,N,N,S,7
3,N,S,N,6
4,S,S,N,3
5,S,N,N,1
6,S,S,S,5
7,S,N,S,4


/home/jovyan/work/esneyder/Documents/ciencia de datos 2020/proyect_minjusticia/MinJusticia_team77_DS4A_Project/app/backend/etl.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Execute ETL SHDI index

In [ ]:
data_shdi = pd.read_excel('../data/SHDI Colombia 4.0.xlsx', index_col='year')

In [ ]:
etl_sdhi = ETL_SDHI(queries)
etl_sdhi.inmate_static_info(data_shdi)

### Encode

In [4]:
encoding = Encoding(queries)
data_2_encode = encoding.get_data()
data_encoded = encoding.one_hot(data_2_encode)
data_encoded

,internoen,genero,nacionalidad,estado_civil,nivel_educativo,reconocimiento_etnico,diversidad_sexual,extranjero,estado_ingreso,fecha_ingreso,...,delito_LESIONES PERSONALES,delito_OTRO,delito_TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,delito_VIOLENCIA INTRAFAMILIAR,tentativa_1,tentativa_2,agravado_1,agravado_2,calificado_1,calificado_2
0,FD8EB8482F09C3F5FD085E4CAD29122368A5937B,2,22,3,2,5,1,1,4,2010-01-02,...,0,0,0,0,1,0,0,1,0,1
1,7624E0648E28D778D1EE4581998B00C3ABC061DA,2,22,3,4,1,1,1,4,2010-01-02,...,0,1,0,0,1,0,1,0,1,0
2,2660D0E0F43523200EF105604266F18D6CA396E5,2,22,3,3,5,1,1,4,2010-01-03,...,0,0,1,0,1,0,1,0,1,0
3,00CD632E133B134F4CD3659080F8E5E7C1C7330A,2,22,3,3,5,1,1,4,2010-01-04,...,0,0,0,1,1,0,1,0,1,0
4,7790E7531D616FDA7825C9CBB5502DD82DF35EB5,2,22,1,5,5,1,1,2,2010-01-05,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103285,C476DC9C78CBC61BC5A3D27699595E916D90E3E2,2,22,6,2,5,1,1,5,2020-06-05,...,0,0,1,0,1,0,1,0,1,0
103286,AEB9E4C6773CBBE7C8424A52AB686E9F14101E73,2,22,7,10,5,1,1,4,2020-06-07,...,0,1,0,0,1,0,1,0,1,0
103287,92FBC965E9DE8CE7CD6AF95F4C460AF9A788CF59,2,22,6,4,5,1,1,4,2020-06-07,...,0,0,0,0,1,0,0,1,0,1
103288,BE6DD3C2939188AAFE3CD16E971D7F6C2BCEBC73,2,22,7,2,5,1,1,5,2020-06-08,...,0,0,0,0,1,0,1,1,1,1
